In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta

In [2]:
cities = ['ABZ', 'ALE', 'AMA', 'AMM', 'ASP', 'BEI', 'BOT', 'BSL', 'DEL', 'EGB',
          'HAD', 'HEL', 'HPB', 'HYY', 'KCE', 'KPZ', 'MAR', 'MHD', 'MLP', 'MUK', 
          'NAN', 'NEU', 'POV', 'SAO', 'SCH', 'SGP', 'UAE', 'VAR', 'VIE', 'WAL', 'ZOT']

In [3]:
def combine_datasets(folder):
    full_df = []

    for c in cities:
        df = pd.read_csv('data/'+folder+'/'+c+'.csv')
        df['station'] = c
        full_df.append(df)

    full_df = pd.concat(full_df)
    full_df = full_df.reset_index(drop=True)
    full_df['id'] = [full_df.station[i] + '-' + str(full_df.date[i]) for i in range(full_df.shape[0])]
    
    return full_df

In [4]:
aerosols_df = combine_datasets('aerosols')
aerosols_df = aerosols_df[['id', 'latitude', 'longitude',
                           'aermr01', 'aermr02', 'aermr03','aermr04', 'aermr05',
                           'aermr06', 'aermr07', 'aermr08',  'aermr09', 'aermr10']]
# exclude 'aermr11' for now so that there is still data for AMA

aerosols_df.to_csv('data/aerosols_data.csv', index=False)

In [5]:
atmospheric_df = combine_datasets('atmospheric')
atmospheric_df = atmospheric_df[['id', 'd2m', 't2m']]

atmospheric_df.to_csv('data/atomospheric_data.csv', index=False)

In [6]:
boundary_layer_height_df = combine_datasets('boundary_layer_height')
boundary_layer_height_df = boundary_layer_height_df[['id', 'blh']]

boundary_layer_height_df.to_csv('data/boundary_layer_height_data.csv', index=False)

In [7]:
gases_df = combine_datasets('gases')
gases_df = gases_df[['id', 'co', 'c5h8', 'no2', 'no', 'so2']]

gases_df.to_csv('data/gases_data.csv', index=False)

In [8]:
slow_access_df = combine_datasets('slow_access')
slow_access_df = slow_access_df[['id', 'nh3', 'crwc', 'c10h16']]
slow_access_df.to_csv('data/slow_access_data.csv', index=False)

In [9]:
n100_df = []

for c in cities:
    df = pd.read_table('data/N100_proxy/'+c+'_N100.dat', sep='\s+', 
                       names=['year', 'month', 'day', 'hour', 'minute', 'n100'])
    
    df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
    
    # AMA and UAE are in UTC, so the times of the measurements need to be adjusted to local time
    if (c == 'AMA'):
        df['time'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])
        df.time -= timedelta(hours=4)
        df = df.drop(columns='time')
    elif (c == 'UAE'):
        df['time'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])
        df.time += timedelta(hours=4)
        df = df.drop(columns='time')
    
    df = df.groupby('date', as_index=False).mean()
    df.date = df.date.dt.date
    df['station'] = c
    n100_df.append(df)

n100_df = pd.concat(n100_df)
n100_df = n100_df.reset_index(drop=True)
n100_df['id'] = [n100_df.station[i] + '-' + str(n100_df.date[i]) for i in range(n100_df.shape[0])]
n100_df = n100_df[['id', 'station', 'date', 'n100']]
n100_df.to_csv('data/n100_data.csv', index=False)

In [21]:
# combining the data sets into one data frame
data = n100_df.merge(aerosols_df, on='id')
data = data.merge(atmospheric_df, on='id')
data = data.merge(boundary_layer_height_df, on='id')
data = data.merge(gases_df, on='id')
data = data.merge(slow_access_df, on='id')

# calculating the relative humidity
td = data.d2m - 273.15
t = data.t2m - 273.15
data['rh'] = 100*(np.exp((17.625*td) / (243.04+td)) / np.exp((17.625*t) / (243.04+t)))

# calculating the day of the year (doy)
data['date'] =  pd.to_datetime(data['date'])
doy = data.date.dt.dayofyear

# adding the sin and cos of the doy to the dataframe
data['doy_sin'] = np.sin((2*np.pi*doy)/doy.max())
data['doy_cos'] = np.cos((2*np.pi*doy)/doy.max())

# scaling latitudes and longitudes to [-1, 1]
data.latitude /= 90
data.longitude /= 180
data['longitude_sin'] = np.sin((2*np.pi*data.longitude))
data['longitude_cos'] = np.cos((2*np.pi*data.longitude))

# reordering columns
data = data[['id', 'station', 'date', 'doy_sin', 'doy_cos', 'latitude', 'longitude_sin', 'longitude_cos', 
            'n100', 'aermr01', 'aermr02', 'aermr03', 'aermr04', 'aermr05', 'aermr06', 'aermr07', 
            'aermr08', 'aermr09', 'aermr10', 'co', 'c5h8', 'c10h16', 'nh3', 'no', 'no2', 'so2', 
            'd2m', 't2m', 'crwc', 'blh', 'rh']]

data.date = pd.to_datetime(data.date)
data = data.dropna(axis=0)
data.to_csv('data/full_data.csv', index=False)
data.head()

,id,station,date,doy_sin,doy_cos,latitude,longitude_sin,longitude_cos,n100,aermr01,...,c10h16,nh3,no,no2,so2,d2m,t2m,crwc,blh,rh
0,ABZ-2012-01-26,ABZ,2012-01-26,0.431673,0.902030,0.561889,0.438057,0.898947,2708.085714,6.752697e-12,...,2.113917e-10,8.448101e-10,1.302913e-08,2.060121e-08,6.464830e-09,266.65186,268.37625,0.0,306.51462,87.716470
1,ABZ-2012-01-27,ABZ,2012-01-27,0.447094,0.894487,0.561889,0.438057,0.898947,2489.175000,4.223335e-12,...,2.499145e-10,1.008734e-09,1.489063e-08,2.199851e-08,7.113044e-09,265.71246,267.83536,0.0,142.03300,85.013445
2,ABZ-2012-01-28,ABZ,2012-01-28,0.462383,0.886680,0.561889,0.438057,0.898947,3484.229167,3.553853e-12,...,4.513248e-10,1.407371e-09,3.310498e-08,1.887175e-08,7.495814e-09,268.29993,269.69257,0.0,152.80160,90.074210
3,ABZ-2012-01-29,ABZ,2012-01-29,0.477536,0.878612,0.561889,0.438057,0.898947,2977.512500,8.665312e-12,...,1.150862e-10,1.466395e-09,4.424891e-09,1.569601e-08,4.929475e-09,267.07495,269.08804,0.0,461.47455,85.875228
4,ABZ-2012-01-30,ABZ,2012-01-30,0.492548,0.870285,0.561889,0.438057,0.898947,2702.463636,1.436837e-11,...,4.446712e-11,1.477080e-09,1.021235e-09,1.345031e-08,5.219232e-09,263.61273,267.95770,0.0,610.29910,71.525093
